# The Plan
Everything we know about the plan.

### Instructions
1. Run all cells! (click on Menu > Cell > Run All Cells)
1. View report at the bottom.

In [ ]:
#planId = "98db70e2-cee5-4e2d-ae15-dca389fa8f41"
planId = "f38f1a4b-49d7-4f08-a9b9-c81b2c39aff6"

In [ ]:
%%capture 
# install packages, setup workspace root
!pip install azure-kusto-notebooks plotly
import os
from azure.kusto.notebooks import utils as akn

# cwd should be workspace root
if os.path.basename(os.getcwd()) == 'devops-pipelines':
    os.chdir(os.pardir)

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net')

In [ ]:
# collect basic plan info
plan_info = akn.Query(
    client, 'VSO', 
    path=os.path.join('devops-pipelines', 'queries', 'run', 'PlanInfo.csl'), 
    params={'OrchestrationId': akn.quote(planId)})

# collect full plan history
what_happened = akn.Query(client, 'VSO',
    path=os.path.join('devops-pipelines', 'queries', 'run', 'WhatHappened.csl'),
    params={'OrchestrationId': akn.quote(planId)})

# fetch data in parallel
akn.run((plan_info, what_happened))

In [ ]:
# draw basic info

# compute relative time stamps
history = what_happened.dataframe
t0 = history['PreciseTimeStamp'].iloc[0]
history['Time'] = history.apply(lambda row: row['PreciseTimeStamp'] - t0, axis=1)
history.OrchestrationId = history.apply(lambda row: row.OrchestrationId[37:], axis=1)

# record critical times
def find_time(message):
    r = history[history.Message.str.startswith(message)]
    if len(r.index) > 0:
        return r['PreciseTimeStamp'].iloc[0]

create_time = find_time('Created plan')
start_time = find_time('Started plan')
end_time = find_time('Completed orchestration with result')
total_duration = end_time - start_time if end_time and start_time else None

import importlib
importlib.reload(akn)
# info will only exist if the plan has started at least one job :(
d = akn.pandas_row_to_dictionary(plan_info.dataframe)
d['create time'] = create_time
d['start time'] = start_time
d['end time'] = end_time
d['total duration'] = total_duration
r = akn.Report()
r.write(akn.to_md_table(d))

from IPython.display import Markdown
Markdown(r.content)

In [ ]:
# SLA analysis
su = akn.quote(d.get('ScaleUnit', ''))
oids = [akn.quote(joid) for joid in d.get('JobOrchestrationIds', [])]
slas = [akn.Query(client, 'VSO', 
        os.path.join('devops-pipelines', 'queries', 'sla', 'SLAVisualization.csl'),
        params=dict(ScaleUnit=su, OrchestrationId=oid)) 
        for oid in oids]
akn.run(slas)

In [ ]:
# draw all slas
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
if not slas:
    print("There are no jobs associated with this plan.")
else:
    number_of_graphs = min(25, len(slas))
    names = [n[37:] for n in d.get('JobOrchestrationIds',[])]
    fig = make_subplots(cols=2, rows=int(math.ceil(number_of_graphs / 2)), 
                        subplot_titles=names,
                        shared_xaxes=True, 
                        vertical_spacing=0.1)

    for i in range(len(slas)):
        df = slas[i].dataframe
        row = int(i / 2) + 1
        col = int(i % 2) + 1
        name = names[i]
        
        df = slas[0].dataframe
        print(row)
        print(col)
        fig.add_trace(go.Bar(x=df.PhaseName, y=df.PercentDifference, name=name), 
                      row=row, col=col)
    fig.update_xaxes(showgrid=False, tickangle=-60, automargin=True)
    fig.update_xaxes(showgrid=True, zeroline=True, automargin=True)
    fig.update_layout(height=150 * number_of_graphs, 
                      width=1000, showlegend=False,
                      title_text="Analysis!")

    fig.show()

In [ ]:
# draw full history
columns_to_ignore = ('source_', 'PreciseTimeStamp')
columns = ['Time'] + [c for c in history.columns if c not in columns_to_ignore and c != 'Time']

from IPython.display import HTML
HTML(history[columns].to_html(index=False))